In [1]:
import pandas as pd
import re
from datetime import datetime
#from soyspacing.countbase import CountSpace

In [2]:
def remove_patterns(text, patterns: list):
    
    assert type(text) is pd.core.series.Series
    assert type(patterns) is list
    
    for pattern in patterns:
        text = text.str.replace(pattern, " ")
    
    return text


def preprocess_lv1(text):
    
    assert type(text) is pd.core.series.Series
    
    text = text.str.lower()
    
    patterns = {
        "[\s]": " ",
        "[.]{2,}": " ",
        #",": "",
        #".": "",
        "아쉬운점\s*\d*": " ",
        "좋은점\s*\d*": " ",
        "글자수\s*\d*자\d*byte": " ",
        "o{2,}": " "
    }
    
    for pattern in patterns:
        text[text.notna()] = text[text.notna()].apply(lambda x: re.sub(pattern, patterns[pattern], x))
    
    return text


def preprocess_lv2(text, only_hangul=False):
    
    assert type(text) is pd.core.series.Series
    
    pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣]" if only_hangul else "[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9.]"
    
    text[text.notna()] = text[text.notna()].apply(lambda x: re.sub(pattern, " ", x))
    text[text.notna()] = text[text.notna()].apply(lambda x: re.sub("\s+", " ", x))
    
    return text.str.strip()


def correct_spacing(train, test, file_path=None):
    
    for series in [train, test]:
        assert type(series) is pd.core.series.Series
        
    if file_path is not None:
        assert file_path[-4:] == ".txt"
    else:
        file_path = "correct_spacing_train_" + datetime.strftime(datetime.today(), "%Y%m%d") + ".txt"
        
    train.to_csv(file_path, index=None, header=None)
    
    model = CountSpace()
    model.train(file_path)
    
    train = train.apply(lambda x: model.correct(x)[0])
    test = test.apply(lambda x: model.correct(x)[0])
    
    return train, test


In [3]:
data = pd.read_csv("../jobkorea_all.csv")
data.columns

Index(['질문', '답변', '조언', '스펙', '평가', '총평', '주소', '회사명', '지원시기', '근무형태',
       '직무분야'],
      dtype='object')

In [4]:
# 질문, 답변, 조언
data["답변"] = preprocess_lv1(data["답변"])
data["답변"] = preprocess_lv2(data["답변"])

data.loc[0, "답변"]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'능동적인 직무 수행의 기회 코리아세븐 유통업에 대한 관심으로 백화점에서 다년간 ar을 경험하고 에서 매장 관리 손익 데이터 분석 발주 등 영업관리 직무를 수행하였습니다. 그러나 제한된 공간 안에서 이뤄지는 영업관리 직무는 활동적으로 업무를 할 수 있는 기회가 적었고 활동적인 업무를 하며 성취감을 얻을 수 있는 업무를 원했던 저에게 아쉬움이 있었습니다. 하지만 코리아세븐의 영업관리 업무는 다양한 상권의 점포를 직접 관리하며 그 상권에 맞는 효율적인 영업 프로세스를 구축하고 회사의 정책을 전달하는 등 능동적이고 활동적인 업무를 할 수 있는 기회라고 생각하였습니다. 이렇게 능동적으로 활발히 업무를 수행할 수 있는 기회인 코리아세븐의 영업관리 직무를 수행함에 있어 하이퍼마켓 영업관리 경험에서 얻었던 소통 데이터 분석 능력을 활용하여 고객과 경영주의 신뢰를 얻고 그것을 기반으로 매출이라는 성과를 도출해 낼 수 있는 최고의 영업사원이 되도록 노력하겠습니다.'

In [5]:
# 스펙
data["스펙"] = data["스펙"].str.split("\n")
data["스펙"] = data["스펙"].apply(lambda x: x[1:-2])

data.loc[0, "스펙"]

['서울4년', '경제', '학점 3.87/4.5']

In [6]:
data.loc[data["스펙"].apply(lambda x: len(x)).idxmax(), "스펙"]

['지방4년',
 '국제무역학과',
 '학점 3.97/4.5',
 '토익 960',
 '토스 Level7',
 '오픽 IH',
 '자격증 3개',
 '해외경험 3회',
 '인턴 1회',
 '수상 1회',
 '동아리 1회',
 '교내활동 1회',
 '사회활동 1회',
 '자원봉사 1회']

In [7]:
data.loc[data["스펙"].apply(lambda x: len(x)).idxmin(), "스펙"]

['고졸']

In [8]:
# 회사명
data["회사명"] = data["회사명"].str.lower()
data["회사명"] = remove_patterns(data["회사명"], patterns=["㈜", "[(주)]"])
data["회사명"].value_counts()

gs리테일            698
삼성전자             632
sk하이닉스           327
농협은행             317
 kt              303
                ... 
   메이필드            1
   미즈호은행           1
위아마그나파워트레인         1
농림수산식품교육문화정보원      1
   한국경제신문          1
Name: 회사명, Length: 1469, dtype: int64

In [10]:
# 직무분야
data["직무분야"] = data["직무분야"].str.split("·").apply(lambda x: " ".join(x))
data["직무분야"].value_counts()

사무 총무 법무          2385
생산관리 공정관리 품질관리    2269
영업관리 지원 영업기획      2045
제품 서비스영업          1822
기획 전략 경영          1489
                  ... 
영화 배급                3
공연 전시 무대 스텝          3
법률 특허 상표             3
광고 시각디자인             2
음악 음향                1
Name: 직무분야, Length: 95, dtype: int64